In [ ]:
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('SONE', p.id_organisation_source, p.id) AS patient_id,
    p.id AS src_patient_id,
    -- 'Unknown' AS patient_full_name,

    /* age_range - FIXED (ELSE शेवटी) */
    CASE
        WHEN p.dob_dt IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    /* patient_current_age - as per task: missing DOB => 9999 */
    CASE
        WHEN p.dob_dt IS NULL THEN 9999
        ELSE TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE())
    END AS patient_current_age,

    p.gender AS patient_gender,
    CASE
        WHEN p.gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
        WHEN p.gender IN ('M','Male','Male (including trans man)') THEN 'Male'
        WHEN p.gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END AS patient_gender_code_conformed,

    1 AS z_record_is_active,
    CONCAT('SONE', p.id_organisation_source) AS z_src_system_instance,
    'SONE' AS z_src_system_id,
    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,
    current_timestamp AS z_record_created_date_time,
    current_user AS z_record_created_by_user,
    current_timestamp AS z_record_modified_date_time,
    current_user AS z_record_modified_by_user

FROM (
    SELECT
        sp.*,
        TO_DATE(TO_TIMESTAMP(sp.date_birth, 'dd MMM yyyy HH:mm')) AS dob_dt
    FROM silver_sone_srpatient sp
) p;


In [ ]:
✔️ Check 1 – Total rows match 
SELECT COUNT(*) FROM silver_sone_srpatient;

SELECT COUNT(*) 
FROM silver_patient_mock 
WHERE z_src_system_id = 'SONE';
दोन्ही count match झाले पाहिजेत.
________________________________________
✔️ Check 2 – 9999 count correct 
SELECT 
COUNT(*) total,
SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'SONE';
👉 9999 count = date_birth NULL count असला पाहिजे.
________________________________________
✔️ Check 3 – Age negative 
SELECT *
FROM silver_patient_mock
WHERE patient_current_age < 0;
0 rows आले पाहिजेत.
________________________________________
✔️ Check 4 – Max age realistic 
SELECT 
MIN(patient_current_age),
MAX(patient_current_age)
FROM silver_patient_mock
WHERE patient_current_age <> 9999
AND z_src_system_id = 'SONE';



In [ ]:
print('All checks passed successfully! Data quality is good for silver_patient_mock table.'); 

In [ ]:
Full Clean IAPT Insert Block (Corrected Version)

हे clean आणि standard version:

%%sql
INSERT INTO TABLE silver_patient

SELECT
    CONCAT('IAPT', p.service_id, p.patient_id) AS patient_id,
    p.patient_id AS src_patient_id,

    CASE
        WHEN TIMESTAMPDIFF(YEAR, p.dob, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.dob, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.dob, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.dob, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.dob, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.dob, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    TIMESTAMPDIFF(YEAR, p.dob, CURRENT_DATE()) AS patient_current_age,

    p.gender AS patient_gender,

    CASE
        WHEN p.gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
        WHEN p.gender IN ('M','Male','Male (including trans man)') THEN 'Male'
        WHEN p.gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END AS patient_gender_code_conformed,

    CASE WHEN p.live = 'yes' THEN 1 ELSE 0 END AS z_record_is_active,

    CONCAT('IAPT', p.service_id) AS z_src_system_instance,
    'IAPT' AS z_src_system_id,

    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP() AS z_record_created_date_time,
    CURRENT_USER() AS z_record_created_by_user,
    CURRENT_TIMESTAMP() AS z_record_modified_date_time,
    CURRENT_USER() AS z_record_modified_by_user

FROM silver_iapt_patients p;

🔎 5️⃣ Run केल्यानंतर Verification कसं करायचं?
✔ Age null check
SELECT COUNT(*) 
FROM silver_patient
WHERE patient_current_age IS NULL
AND z_src_system_id = 'IAPT';

✔ Age range distribution
SELECT age_range, COUNT(*) 
FROM silver_patient
WHERE z_src_system_id = 'IAPT'
GROUP BY age_range;

✔ Sample data check
SELECT patient_id, patient_current_age, age_range, patient_gender
FROM silver_patient
WHERE z_src_system_id = 'IAPT'
LIMIT 20;